# UCS Search

## Main Attempt (BackTrack)

### Method 1

In [3]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost): # Declare a new Cell
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other): # Less than operator overloading, Used to compare 2 Cell objects by Cost
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0, 0), goal=(-1,-1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] # -1 = Safeguard, -2 = Obsticle, -3 = Wall, -4 = Animal
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal


        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate) # Get number of cells to overwrite

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)] # Get indices of cells that aren't starting or goal cell

        chosen_indices = random.sample(indices, num_chosen_cells) # Get random cells indices

        for i, j in chosen_indices:
            self.grid[i][j] = symbol # Overwrite cell with new symbol

    
    def is_valid_move(self, x, y): # Check to see if move is valid
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node): # Get valid neighbors for a cell

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell): # Calculate cost of a cell
        if cell == -1: # Safeguard
            return random.randint(1, 3)
        elif cell == -2: # Obsticle
            return random.randint(2, 4)
        elif cell == -3: # Wall
            return 1
        elif cell == -4: # Animal
            if random.random() < 0.8: # Killed
                return random.randint(2, 4)
            else: # Survived
                return random.randint(1, 3)
        else:
            return 0 

    
    def ucs_treasure_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set() # Used to store all explored cells

        while queue:
            current_cost, current_node = heapq.heappop(queue) # Pop from priority queue

            if (current_node.x, current_node.y) == self.goal: # If current cell is goal cell 
                print('Total Cost:', current_cost)
                return self.construct_path(current_node)
            
            if current_node.cost == -3: # If Current cell is wall
                    if current_node.parent and current_node.parent.parent: # If parent of parent exists
                        current_node = current_node.parent.parent # Go back 2 steps
                        current_cost += 2
                        continue
                    else:
                        #continue
                        if current_node.parent: # If parent exits
                            current_node = current_node.parent # Go back 1 step
                            current_cost += 1
                            continue
                        else:
                            continue

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node): # Get neighbors

                n_cost = self.calculate_cost(neighbor.cost)
                new_cost = current_cost + n_cost # Calculate cost of neighbor

                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node # Keep track of parents

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
        
            # self.display(self.construct_path(current_node))


        print('Got Stuck')
    
    
    def construct_path(self, node): # Construct a path from goal to start
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path): # Display the maze

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()




forest = Forest(8, animal_rate=0.1, wall_rate=0.1, obs_rate=0.1)
path = forest.ucs_treasure_search()
print("Optimal Path:")
print(path)
print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 24
Optimal Path:
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4), (4, 5), (4, 6), (4, 7), (5, 7), (6, 7), (7, 7)]

Enchanted Forest Grid:

Original Maze:
2 - - 0 0 A 0 5 
0 0 0 - - 0 - A 
0 A 0 0 0 0 0 0 
0 0 A 0 0 0 0 0 
0 A 0 5 0 0 0 0 
- 0 0 0 5 0 5 0 
0 0 0 0 0 5 0 0 
5 0 0 0 0 0 0 3 



Traversed Maze:
2 - - 0 0 A 0 5 
* * * - - 0 - A 
0 A * * * 0 0 0 
0 0 A 0 * 0 0 0 
0 A 0 5 * * * * 
- 0 0 0 5 0 5 * 
0 0 0 0 0 5 0 * 
5 0 0 0 0 0 0 3 


### Method 2

In [57]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost): # Declare a new Cell
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other): # Less than operator overloading, Used to compare 2 Cell objects by Cost
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0, 0), goal=(-1,-1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] # -1 = Safeguard, -2 = Obsticle, -3 = Wall, -4 = Animal
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal


        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate) # Get number of cells to overwrite

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)] # Get indices of cells that aren't starting or goal cell

        chosen_indices = random.sample(indices, num_chosen_cells) # Get random cells indices

        for i, j in chosen_indices:
            self.grid[i][j] = symbol # Overwrite cell with new symbol

    
    def is_valid_move(self, x, y): # Check to see if move is valid
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node): # Get valid neighbors for a cell

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell): # Calculate cost of a cell
        if cell == -1: # Safeguard
            return random.randint(1, 3)
        elif cell == -2: # Obsticle
            return random.randint(2, 4)
        elif cell == -3: # Wall
            return 1
        elif cell == -4: # Animal
            if random.random() < 0.8: # Killed
                return random.randint(2, 4)
            else: # Survived
                return random.randint(1, 3)
        else:
            return 0 

    
    def ucs_treasure_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set() # Used to store all explored cells
        all_costs = {(self.start[0], self.start[1]): 0}

        while queue:
            current_cost, current_node = heapq.heappop(queue) # Pop from priority queue

            if (current_node.x, current_node.y) == self.goal: # If current cell is goal cell 
                # print('Total Cost:', current_cost)
                print('Total Cost:', all_costs[(current_node.x, current_node.y)], end='\n\n')
                
                return self.construct_path(current_node), all_costs
            
            if current_node.cost == -3: # If Current cell is wall
                    if current_node.parent and current_node.parent.parent: # If parent of parent exists
                        current_node = current_node.parent.parent # Go back 2 steps
                        current_cost += 2
                        continue
                    else:
                        #continue
                        if current_node.parent: # If parent exits
                            current_node = current_node.parent # Go back 1 step
                            current_cost += 1
                            continue
                        else:
                            continue

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node): # Get neighbors

                n_cost = self.calculate_cost(neighbor.cost)

                if (neighbor.x, neighbor.y) not in all_costs:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost
                
                elif (all_costs[(current_node.x, current_node.y)] + n_cost) < all_costs[(neighbor.x, neighbor.y)]:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost

                # new_cost = current_cost + n_cost # Calculate cost of neighbor
                new_cost = all_costs[(neighbor.x, neighbor.y)] # Calculate cost of neighbor


                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node # Keep track of parents

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
        
            # self.display(self.construct_path(current_node))


        print('Got Stuck')
    
    
    def construct_path(self, node): # Construct a path from goal to start
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path): # Display the maze

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()




forest = Forest(8, animal_rate=0.1, wall_rate=0.1, obs_rate=0.1)
path, costs = forest.ucs_treasure_search()
print("Optimal Path:")
print(path)

if path != None:
    print('Cost per Step:')
    for key in costs:
        if key in path:
            print('{}:{}'.format(key, costs[key]), end=' , ')

print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 16

Optimal Path:
[(0, 0), (0, 1), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (6, 4), (7, 4), (7, 5), (7, 6), (7, 7)]
Cost per Step:
(0, 0):0 , (0, 1):1 , (1, 1):3 , (2, 1):5 , (3, 1):6 , (3, 2):7 , (3, 3):8 , (4, 3):9 , (4, 4):10 , (5, 4):11 , (6, 4):12 , (7, 4):13 , (7, 5):14 , (7, 6):15 , (7, 7):16 , 
Enchanted Forest Grid:

Original Maze:
2 0 A 0 - 0 5 0 
0 - 5 0 0 0 0 0 
A 0 0 0 0 0 0 - 
0 0 0 0 - 0 5 A 
0 0 - 0 0 0 0 - 
0 0 0 0 0 0 5 A 
0 0 0 0 0 0 0 5 
0 0 0 A 0 0 0 3 



Traversed Maze:
2 * A 0 - 0 5 0 
0 * 5 0 0 0 0 0 
A * 0 0 0 0 0 - 
0 * * * - 0 5 A 
0 0 - * * 0 0 - 
0 0 0 0 * 0 5 A 
0 0 0 0 * 0 0 5 
0 0 0 A * * * 3 


## No BackTrack

### Method 1

In [4]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other):
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0,0), goal=(-1, -1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal
            
        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate)

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)]

        chosen_indices = random.sample(indices, num_chosen_cells)

        for i, j in chosen_indices:
            self.grid[i][j] = symbol

    
    def is_valid_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node):

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell):
        if cell == -1:
            return random.randint(1, 3)
        elif cell == -2:
            return random.randint(2, 4)
        elif cell == -3:
            return float('inf')
        elif cell == -4:
            if random.random() < 0.8:
                return random.randint(2, 4)
            else:
                return random.randint(1, 3)
        else:
            return 0

    
    def ucs_treasure_search(self):

        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set()

        while queue:
            current_cost, current_node = heapq.heappop(queue)

            if (current_node.x, current_node.y) == self.goal:
                print('Total Cost:', current_cost)
                return self.construct_path(current_node)

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node):

                if neighbor.cost == -3: # If wall encountered, skip
                    # current_cost += 2
                    continue

                n_cost = self.calculate_cost(neighbor.cost)
                new_cost = current_cost + n_cost

                if (neighbor.x, neighbor.y) not in explored:
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
            
            # self.display(self.construct_path(current_node))


        print('Got Stuck')

    
    def construct_path(self, node):
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path):

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()


        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()



forest = Forest(8, animal_rate=0.1, wall_rate=0.2, obs_rate=0.1)
path= forest.ucs_treasure_search()

print("Optimal Path:")
print(path)

print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 26
Optimal Path:
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (3, 2), (3, 3), (3, 4), (4, 4), (4, 5), (4, 6), (5, 6), (5, 7), (6, 7), (7, 7)]

Enchanted Forest Grid:

Original Maze:
2 5 0 0 0 0 5 0 
0 - 0 5 A 0 0 - 
5 5 0 - 0 5 0 A 
5 0 - 0 0 0 0 - 
0 0 0 5 0 0 0 0 
0 0 0 0 0 5 0 0 
0 - 0 0 5 0 5 0 
0 A A 5 0 0 A 3 



Traversed Maze:
2 5 0 0 0 0 5 0 
* * * 5 A 0 0 - 
5 5 * - 0 5 0 A 
5 0 * * * 0 0 - 
0 0 0 5 * * * 0 
0 0 0 0 0 5 * * 
0 - 0 0 5 0 5 * 
0 A A 5 0 0 A 3 


## Method 2

In [60]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other):
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0,0), goal=(-1, -1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal
            
        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate)

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)]

        chosen_indices = random.sample(indices, num_chosen_cells)

        for i, j in chosen_indices:
            self.grid[i][j] = symbol

    
    def is_valid_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node):

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell):
        if cell == -1:
            return random.randint(1, 3)
        elif cell == -2:
            return random.randint(2, 4)
        elif cell == -3:
            return float('inf')
        elif cell == -4:
            if random.random() < 0.8:
                return random.randint(2, 4)
            else:
                return random.randint(1, 3)
        else:
            return 0

    
    def ucs_treasure_search(self):

        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set()
        all_costs = {(self.start[0], self.start[1]): 0}

        while queue:
            current_cost, current_node = heapq.heappop(queue)

            if (current_node.x, current_node.y) == self.goal:
                # print('Total Cost:', current_cost)
                print('Total Cost:', all_costs[(current_node.x, current_node.y)], end='\n\n')
                return self.construct_path(current_node), all_costs

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node):

                if neighbor.cost == -3: # If wall encountered, skip
                    # current_cost += 2
                    continue

                n_cost = self.calculate_cost(neighbor.cost)

                if (neighbor.x, neighbor.y) not in all_costs:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost
                
                elif (all_costs[(current_node.x, current_node.y)] + n_cost) < all_costs[(neighbor.x, neighbor.y)]:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost

                # new_cost = current_cost + n_cost
                new_cost = all_costs[(neighbor.x, neighbor.y)]  # Calculate cost of neighbor

                if (neighbor.x, neighbor.y) not in explored:
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
            
            # self.display(self.construct_path(current_node))


        print('Got Stuck')

    
    def construct_path(self, node):
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path):

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()


        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()



forest = Forest(8, animal_rate=0.1, wall_rate=0.2, obs_rate=0.1)
path, costs = forest.ucs_treasure_search()

print("Optimal Path:")
print(path)

if path != None:
    print('Cost per Step:')
    for key in costs:
        if key in path:
            print('{}:{}'.format(key, costs[key]), end=' , ')

print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 20

Optimal Path:
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (4, 6), (4, 7), (5, 7), (6, 7), (7, 7)]
Cost per Step:
(0, 0):0 , (0, 1):2 , (0, 2):5 , (0, 3):6 , (1, 3):8 , (2, 3):10 , (2, 4):11 , (2, 5):12 , (3, 5):14 , (4, 5):15 , (4, 6):16 , (4, 7):17 , (5, 7):19 , (6, 7):19 , (7, 7):20 , 
Enchanted Forest Grid:

Original Maze:
2 - A 0 5 5 0 - 
0 5 5 - 5 0 0 - 
0 0 5 - 0 0 5 5 
0 0 0 0 A 0 0 0 
0 5 0 0 A 0 0 0 
0 0 0 0 0 5 0 - 
A 0 0 0 5 0 0 0 
0 0 0 0 0 5 0 3 



Traversed Maze:
2 * * * 5 5 0 - 
0 5 5 * 5 0 0 - 
0 0 5 * * * 5 5 
0 0 0 0 A * 0 0 
0 5 0 0 A * * * 
0 0 0 0 0 5 0 * 
A 0 0 0 5 0 0 * 
0 0 0 0 0 5 0 3 


# A* Search

## Main Attempt (BackTrack)

### Method 1

In [34]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost): # Declare a new Cell
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other): # Less than operator overloading, Used to compare 2 Cell objects by Cost
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0, 0), goal=(-1,-1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal


        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate) # Get number of cells to overwrite

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)] # Get indices of cells that aren't starting or goal cell

        chosen_indices = random.sample(indices, num_chosen_cells) # Get random cells indices

        for i, j in chosen_indices:
            self.grid[i][j] = symbol # Overwrite cell with new symbol

    
    def is_valid_move(self, x, y): # Check to see if move is valid
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node): # Get valid neighbors for a cell

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell): # Calculate cost of a cell 
        if cell == -1: # Safeguard
            return random.randint(1, 3)
        elif cell == -2: # Obsticle
            return random.randint(2, 4)
        elif cell == -3: # Wall
            return 1
        elif cell == -4: # Animal 
            if random.random() < 0.8: # Killed
                return random.randint(2, 4)
            else: # Survived
                return random.randint(1, 3)
        else:
            return 0

    
    def a_star_treasure_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set() # Used to store all explored cells

        while queue:
            current_cost, current_node = heapq.heappop(queue) # Pop from priority queue

            if (current_node.x, current_node.y) == self.goal: # If current cell is goal cell 
                print('Total Cost:', current_cost)
                return self.construct_path(current_node)
            
            # If encounter a Wall, move to parent
            if current_node.cost == -3: # If Current cell is wall
                    if current_node.parent and current_node.parent.parent: # if parent of parent exists
                        current_node = current_node.parent.parent # Go back 2 steps
                        current_cost += 2
                        continue
                    else:
                        #continue
                        if current_node.parent: # if only parent exists
                            current_node = current_node.parent # Go back 1 step
                            current_cost += 1
                            continue
                        else:
                            continue

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node): # Get neighbors

                n_cost = self.calculate_cost(neighbor.cost)
                
                if (current_node.x, current_node.y) != self.start:
                    new_cost = current_cost + n_cost + self.heuristics(neighbor) - self.heuristics(current_node) # Calculate cost of neighbor by subtracting previous heuristics estimate from current
                else:
                    new_cost = current_cost + n_cost + self.heuristics(neighbor) # Calculate cost of neighbor

                # new_cost = current_cost + n_cost + self.heuristics(neighbor)

                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node # Keep track of parents

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
        
            # self.display(self.construct_path(current_node))


        print('Got Stuck')
    

    def heuristics(self, cell): # Calculate estimated cost by checking displacement from Goal Cell
        return abs( self.goal[0] - cell.x ) +  abs( self.goal[1] - cell.y ) # Manhattan distance

    
    def construct_path(self, node): # Construct a path from goal to start
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path): # Display the maze

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()




forest = Forest(8, animal_rate=0.1, wall_rate=0.1, obs_rate=0.1)
path = forest.a_star_treasure_search()
print("Optimal Path:")
print(path)
print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 19
Optimal Path:
[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (3, 4), (4, 4), (5, 4), (5, 5), (6, 5), (6, 6), (7, 6), (7, 7)]

Enchanted Forest Grid:

Original Maze:
2 A 0 A 0 0 0 5 
A A - 0 - 0 0 0 
0 0 0 0 0 0 0 0 
5 0 0 0 0 - - - 
0 0 A 0 0 0 0 0 
0 0 A 0 0 0 0 0 
0 0 5 0 0 0 0 5 
0 0 5 0 - 5 0 3 



Traversed Maze:
2 A 0 A 0 0 0 5 
* A - 0 - 0 0 0 
* * * 0 0 0 0 0 
5 0 * * * - - - 
0 0 A 0 * 0 0 0 
0 0 A 0 * * 0 0 
0 0 5 0 0 * * 5 
0 0 5 0 - 5 * 3 


## Method 2

In [55]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost): # Declare a new Cell
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other): # Less than operator overloading, Used to compare 2 Cell objects by Cost
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0, 0), goal=(-1,-1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal


        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate) # Get number of cells to overwrite

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)] # Get indices of cells that aren't starting or goal cell

        chosen_indices = random.sample(indices, num_chosen_cells) # Get random cells indices

        for i, j in chosen_indices:
            self.grid[i][j] = symbol # Overwrite cell with new symbol

    
    def is_valid_move(self, x, y): # Check to see if move is valid
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node): # Get valid neighbors for a cell

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell): # Calculate cost of a cell 
        if cell == -1: # Safeguard
            return random.randint(1, 3)
        elif cell == -2: # Obsticle
            return random.randint(2, 4)
        elif cell == -3: # Wall
            return 1
        elif cell == -4: # Animal 
            if random.random() < 0.8: # Killed
                return random.randint(2, 4)
            else: # Survived
                return random.randint(1, 3)
        else:
            return 0

    
    def a_star_treasure_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set() # Used to store all explored cells
        all_costs = {(self.start[0], self.start[1]): 0}

        while queue:
            current_cost, current_node = heapq.heappop(queue) # Pop from priority queue

            if (current_node.x, current_node.y) == self.goal: # If current cell is goal cell 
                print('Total Cost:', all_costs[(current_node.x, current_node.y)], end='\n\n')
                return self.construct_path(current_node), all_costs
            
            # If encounter a Wall, move to parent
            if current_node.cost == -3: # If Current cell is wall
                    if current_node.parent and current_node.parent.parent: # if parent of parent exists
                        current_node = current_node.parent.parent # Go back 2 steps
                        current_cost += 2
                        continue
                    else:
                        #continue
                        if current_node.parent: # if only parent exists
                            current_node = current_node.parent # Go back 1 step
                            current_cost += 1
                            continue
                        else:
                            continue

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node): # Get neighbors

                n_cost = self.calculate_cost(neighbor.cost)

                if (neighbor.x, neighbor.y) not in all_costs:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost
                
                elif (all_costs[(current_node.x, current_node.y)] + n_cost) < all_costs[(neighbor.x, neighbor.y)]:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost

                # new_cost = current_cost + n_cost + self.heuristics(neighbor) # Calculate cost of neighbor
                new_cost = all_costs[(neighbor.x, neighbor.y)]  + self.heuristics(neighbor) # Calculate cost of neighbor

                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node # Keep track of parents

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
        
            # self.display(self.construct_path(current_node))


        print('Got Stuck')
    

    def heuristics(self, cell): # Calculate estimated cost by checking displacement from Goal Cell
        return abs( self.goal[0] - cell.x ) +  abs( self.goal[1] - cell.y ) # Manhattan distance

    
    def construct_path(self, node): # Construct a path from goal to start
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path): # Display the maze

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()




forest = Forest(8, animal_rate=0.1, wall_rate=0.1, obs_rate=0.1)
path, costs = forest.a_star_treasure_search()
print("Optimal Path:")
print(path)

if path != None:
    print('Cost per Step:')
    for key in costs:
        if key in path:
            print('{}:{}'.format(key, costs[key]), end=' , ')

print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 23

Optimal Path:
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (6, 1), (7, 1), (7, 2), (7, 3), (7, 4), (6, 4), (6, 5), (7, 5), (7, 6), (6, 6), (6, 7), (7, 7)]
Cost per Step:
(0, 0):0 , (0, 1):2 , (1, 1):3 , (2, 1):4 , (2, 0):5 , (3, 0):7 , (4, 0):8 , (5, 0):10 , (5, 1):11 , (6, 1):12 , (7, 1):13 , (7, 2):16 , (6, 4):18 , (7, 3):17 , (6, 5):19 , (7, 4):18 , (7, 5):20 , (6, 6):21 , (7, 6):21 , (7, 7):23 , (6, 7):22 , 
Enchanted Forest Grid:

Original Maze:
2 0 5 0 0 A 0 0 
- 0 - 0 0 5 0 0 
0 0 0 0 0 0 A - 
- 5 0 5 0 0 A - 
0 0 0 A 0 0 0 0 
0 0 0 0 0 0 0 - 
0 0 5 5 0 0 0 0 
0 0 0 0 0 0 0 3 



Traversed Maze:
2 * 5 0 0 A 0 0 
- * - 0 0 5 0 0 
* * 0 0 0 0 A - 
* 5 0 5 0 0 A - 
* 0 0 A 0 0 0 0 
* * 0 0 0 0 0 - 
0 * 5 5 * * * * 
0 * * * * * * 3 


## No BackTrack

### Method 1

In [39]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other):
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0,0), goal=(-1, -1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal
            
        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate)

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)]

        chosen_indices = random.sample(indices, num_chosen_cells)

        for i, j in chosen_indices:
            self.grid[i][j] = symbol

    
    def is_valid_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node):

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell):
        if cell == -1:
            return random.randint(1, 3)
        elif cell == -2:
            return random.randint(2, 4)
        elif cell == -3:
            return float('inf')
        elif cell == -4:
            if random.random() < 0.8:
                return random.randint(2, 4)
            else:
                return random.randint(1, 3)
        else:
            return 0

    
    def a_star_treasure_search(self):

        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set()

        while queue:
            current_cost, current_node = heapq.heappop(queue)

            if (current_node.x, current_node.y) == self.goal:
                print('Total Cost:', current_cost)
                return self.construct_path(current_node)

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node):

                if neighbor.cost == -3: # If wall encountered, skip
                    # current_cost += 2
                    continue

                n_cost = self.calculate_cost(neighbor.cost)

                if (current_node.x, current_node.y) != self.start:
                    new_cost = current_cost + n_cost + self.heuristics(neighbor) - self.heuristics(current_node) # Calculate cost of neighbor by subtracting previous heuristics estimate from current
                else:
                    new_cost = current_cost + n_cost + self.heuristics(neighbor) # Calculate cost of neighbor

                # new_cost = current_cost + n_cost + self.heuristics(neighbor)

                if (neighbor.x, neighbor.y) not in explored:
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
            
            # self.display(self.construct_path(current_node))


        print('Got Stuck')


    def heuristics(self, cell):
        return abs( self.goal[0] - cell.x ) +  abs( self.goal[1] - cell.y )

    
    def construct_path(self, node):
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path):

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()



forest = Forest(8, animal_rate=0.1, wall_rate=0.2, obs_rate=0.1)
path= forest.a_star_treasure_search()

print("Optimal Path:")
print(path)

print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 24
Optimal Path:
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (4, 2), (5, 2), (5, 3), (5, 4), (6, 4), (7, 4), (7, 5), (7, 6), (7, 7)]

Enchanted Forest Grid:

Original Maze:
2 5 0 A 0 0 0 0 
0 0 0 0 0 0 5 0 
- - A 0 0 0 A 5 
0 0 - 0 0 0 0 0 
0 - 0 0 5 5 0 0 
0 0 0 A 0 5 0 5 
0 - 0 0 0 5 - 0 
A 0 0 5 0 0 0 3 



Traversed Maze:
2 5 0 A 0 0 0 0 
* 0 0 0 0 0 5 0 
* - A 0 0 0 A 5 
* * - 0 0 0 0 0 
0 * * 0 5 5 0 0 
0 0 * * * 5 0 5 
0 - 0 0 * 5 - 0 
A 0 0 5 * * * 3 


## Method 2

In [54]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other):
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0,0), goal=(-1, -1), animal_rate=0.1, wall_rate=0.1, obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal
            
        self.grid[self.start[0]][self.start[1]] = -1  # Set start position cost
        self.grid[self.goal[0]][self.goal[1]] = -1    # Set goal position cost

        self.mark_cells(animal_rate, -4) # animal
        self.mark_cells(wall_rate, -3) # wall
        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): # Overwrites Cells with new Symbols
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate)

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)]

        chosen_indices = random.sample(indices, num_chosen_cells)

        for i, j in chosen_indices:
            self.grid[i][j] = symbol

    
    def is_valid_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node):

        neighbors = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
    
   
    def calculate_cost(self, cell):
        if cell == -1:
            return random.randint(1, 3)
        elif cell == -2:
            return random.randint(2, 4)
        elif cell == -3:
            return float('inf')
        elif cell == -4:
            if random.random() < 0.8:
                return random.randint(2, 4)
            else:
                return random.randint(1, 3)
        else:
            return 0

    
    def a_star_treasure_search(self):

        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set()
        all_costs = {(self.start[0], self.start[1]): 0}

        while queue:
            current_cost, current_node = heapq.heappop(queue)

            if (current_node.x, current_node.y) == self.goal:
                print('Total Cost:', all_costs[(current_node.x, current_node.y)], end='\n\n')
                return self.construct_path(current_node), all_costs

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node):

                if neighbor.cost == -3: # If wall encountered, skip
                    # current_cost += 2
                    continue

                n_cost = self.calculate_cost(neighbor.cost)

                if (neighbor.x, neighbor.y) not in all_costs:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost
                
                elif (all_costs[(current_node.x, current_node.y)] + n_cost) < all_costs[(neighbor.x, neighbor.y)]:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost

                # new_cost = current_cost + n_cost + self.heuristics(neighbor)
                new_cost = all_costs[(neighbor.x, neighbor.y)]  + self.heuristics(neighbor) # Calculate cost of neighbor

                if (neighbor.x, neighbor.y) not in explored:
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node

            # print('\n\n Curr:', current_node.x, current_node.y, current_cost, '\n\n')
            
            # self.display(self.construct_path(current_node))


        print('Got Stuck')


    def heuristics(self, cell):
        return abs( self.goal[0] - cell.x ) +  abs( self.goal[1] - cell.y )

    
    def construct_path(self, node):
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path):

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('2', end=' ')
                elif (i, j) == self.goal:
                    print('3', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                elif self.grid[i][j] == -3:
                    print('5', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()



forest = Forest(8, animal_rate=0.1, wall_rate=0.2, obs_rate=0.1)
path, costs = forest.a_star_treasure_search()

print("Optimal Path:")
print(path)

if path != None:
    print('Cost per Step:')
    for key in costs:
        if key in path:
            print('{}:{}'.format(key, costs[key]), end=' , ')

print("\nEnchanted Forest Grid:")
forest.display(path)


Total Cost: 25

Optimal Path:
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (6, 6), (6, 7), (7, 7)]
Cost per Step:
(0, 0):0 , (0, 1):1 , (0, 2):2 , (0, 3):5 , (0, 4):6 , (1, 4):7 , (1, 5):8 , (2, 5):10 , (3, 5):11 , (4, 5):13 , (5, 5):14 , (6, 5):17 , (6, 6):20 , (6, 7):22 , (7, 7):25 , 
Enchanted Forest Grid:

Original Maze:
2 0 0 0 0 A 5 0 
5 0 5 - 0 0 A 0 
0 - 0 A 0 0 A 0 
0 0 5 5 0 0 0 0 
0 - 0 5 0 0 0 5 
5 0 0 0 A 0 5 0 
0 0 0 - 5 0 0 0 
- - 0 5 0 0 5 3 



Traversed Maze:
2 * * * * A 5 0 
5 0 5 - * * A 0 
0 - 0 A 0 * A 0 
0 0 5 5 0 * 0 0 
0 - 0 5 0 * 0 5 
5 0 0 0 A * 5 0 
0 0 0 - 5 * * * 
- - 0 5 0 0 5 3 
